In [97]:
# !pip install selenium

# Scraping Job info from LinkedIn

<a href="https://medium.com/nerd-for-tech/linked-in-web-scraper-using-selenium-15189959b3ba">Tutorial here.</a>


In [98]:
import os
import time
import logging

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

driver_path = "geckodriver.exe"
driver = webdriver.Chrome(executable_path=driver_path)

C:\Users\balde\AppData\Local\Temp\ipykernel_6688\343486925.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)


## Authentication

In [212]:
# Set up environment variables in Powershell:

# $Env:Email = "email"
# echo $Env:Email

In [215]:
delay = 5

# set credentials:
email = os.getenv('Email')
password = os.getenv('Password')
print(email)

None


In [189]:
# Go to linkedin and login
driver.get('https://www.linkedin.com/login')

# TODO: Use event listeners
# wait for the browser to load elements
time.sleep(delay)

driver.find_element(By.ID, "username").send_keys(email)
driver.find_element(By.ID, "password").send_keys(password, Keys.ENTER)

NameError: name 'os' is not defined

## Search jobs by title and location

In [100]:
job_title = "data science"
job_location = "canada"

In [101]:
# driver.get("https://www.linkedin.com/jobs/")
# time.sleep(delay)
#
# # find the keyword search bars, then fill it
# search_by_keyword_bar = driver.find_element(By.ID, "jobs-search-box-keyword-id-ember22")
# search_by_keyword_bar.send_keys(job_title)
#
# # TODO use listener instead of 3 seconds sleep
# # find the location search bars, fill it, enter
# time.sleep(delay)
# search_by_location_bar = driver.find_element(By.ID, "jobs-search-box-location-id-ember22")
# search_by_location_bar.send_keys(job_location, Keys.RETURN)

### Alternative

In [126]:
def wait_till_element_ready(by, selector):
    """
    Wait for a given element to be ready

    :param by: the selector strategy. E.g.: By.ID, By.CLASS_NAME, etc.
    :param selector: the CSS selector according to the "By" strategy. E.g.: 'class name', 'id', etc.
    :return:
    """

    colors = {'WARNING': '\033[93m', 'ERROR': '\033[91m'}
    try:
        WebDriverWait(driver, 8).until(EC.presence_of_element_located((by, selector)))
        time.sleep(delay)
    except TimeoutException:
        print(colors['WARNING'] + "Warning: Element has not been loaded!")
        pass

In [122]:
driver.get("https://www.linkedin.com/jobs/")
# time.sleep(delay)

# find the two search boxes: title and location
wait_till_element_ready(By.CLASS_NAME, "jobs-search-box__text-input")
search_bars = driver.find_elements(By.CLASS_NAME, "jobs-search-box__text-input")
print('Found', len(search_bars), 'search_bars')

# enter job title in the search  by keyword bar
search_by_keyword_bar = search_bars[0]
search_by_keyword_bar.send_keys(job_title)

# enter job location in the search by location bar
search_by_location_bar = search_bars[3]
search_by_location_bar.send_keys(job_location, Keys.RETURN)

Found 5 search_bars


## Get the Jobs Info

In [123]:
def get_job_skills():

    # click display skills popup
    wait_till_element_ready(By.CLASS_NAME, 'jobs-unified-top-card__job-insight-text-button')
    # time.sleep(delay)
    view_skills_button = driver.find_element(By.CLASS_NAME, 'jobs-unified-top-card__job-insight-text-button')
    view_skills_button.click()

    # get skills list
    wait_till_element_ready(By.CLASS_NAME, "job-details-skill-match-status-list")
    # time.sleep(delay)
    skills_list = driver.find_element(By.CLASS_NAME, 'job-details-skill-match-status-list')

    # get clean skills list
    skills_list_clean = skills_list.text.split('\n')
    skills_list_clean = list(filter(lambda skill: skill != 'Add', skills_list_clean))

    # close the skills list popup
    wait_till_element_ready(By.CLASS_NAME, 'artdeco-modal__dismiss')
    # time.sleep(delay)
    close_popup_button = driver.find_element(By.CLASS_NAME, 'artdeco-modal__dismiss')
    close_popup_button.click()

    return skills_list_clean


In [124]:
# Get the results of the first page
wait_till_element_ready(By.CLASS_NAME, "jobs-search-results__list-item")
# time.sleep(delay)
jobs_list = driver.find_elements(By.CLASS_NAME, "jobs-search-results__list-item")

jobs = []
for job in jobs_list:

    time.sleep(delay)
    # scroll the job div into view
    driver.execute_script("arguments[0].scrollIntoView();", job)
    job.click()
    # time.sleep(delay)

    # get the position company, location, and remote/on-site
    try:
        [position, company, location, work_mode] = job.text.split('\n')[:4]
        break
    except ValueError:
        position = job.text.split('\n')
        [company, location, work_mode] = ''
        print('job.text.split Error', job.text.split('\n'))

    # get skills list for the job
    skills = get_job_skills()

    # description
    description = driver.find_elements(By.CLASS_NAME, "jobs-description-content__text")

    # all the details
    # details = driver.find_element(By.ID, "job-details").text

    # do what you want with that info...
    jobs.append([position, company, location, work_mode, skills, description])

Issue: Element has not been loaded!
Clean skills list: ['Batch Processing', 'Big Data', 'Computer Science', 'Data Architecture', 'Data Governance', 'Data Integration', 'Data Modeling', 'Extract, Transform, Load (ETL)', 'Informatica', 'Statutory Accounting Principles (SAP)']
Issue: Element has not been loaded!
Clean skills list: ['Data Science', 'Machine Learning', 'Computer Science', 'Human Computer Interaction', 'Natural Language Processing (NLP)', 'Pattern Recognition', 'Polymerase Chain Reaction (PCR)', 'Programming', 'Programming Languages', 'Team Motivation']
Issue: Element has not been loaded!
Clean skills list: ['Commercial Software', 'Communication', 'IT Operations', 'Information Technology', 'Mobile Communications', 'Printers', 'Problem Solving', 'Queues', 'Receptions', 'Repositories']
Issue: Element has not been loaded!
Clean skills list: ['Computer Hardware', 'Computer Information Systems', 'Information Technology', 'Interpersonal Communication', 'Laptops', 'Live Streaming',

ValueError: not enough values to unpack (expected 4, got 1)

Profile - Profil recherché - About You
Skills - Requirements - Position Requirements - Qualifications - Basic Qualifications - Expérience - Éxigences - Compétences - QUALITÉS
Atouts - Preferred Qualifications - Autres exigences

In [114]:
jobs[0][4]

'SQL\nCommunication\nAdd\nDrones\nAdd\nFull-Stack Development\nAdd\nHTML\nAdd\nInformation Technology\nAdd\nJavaScript\nAdd\nNetworking\nAdd\nProcedural Programming\nAdd\nTechnical Support\nAdd'

# Connect to SQL Server

In [154]:
# !pip install pypyodbc

In [177]:
import pypyodbc as odbc

DRIVER_NAME = 'SQL Server'
SERVER_NAME = '.'
DATABASE_NAME = 'LinkedIn'

connection_string = f"""
    DRIVER={{{DRIVER_NAME}}};
    SERVER={{{SERVER_NAME}}};
    DATABASE={{{DATABASE_NAME}}};
    Trust_Connection=yes;
"""

conn = odbc.connect(connectString=connection_string)
print(conn)

In [ ]:
# # Execute a sample query
# cursor.execute('SELECT * FROM your_table_name')
# # Fetch the results of the query
# rows = cursor.fetchall()
# # Print the results
# for row in rows:
#     print(row)

### Insert jobs into the database

In [187]:
def save_jobs(jobs):
    """
    Inserts the jobs into the databse

    :param jobs: jobs to be inserted
    :return:
    """
    insert_query = "INSERT INTO Jobs (position, company, location, work_mode, skills, details) VALUES (?, ?, ?, ?, ?, ?)"

    for job in jobs:
        position = job[0]
        company = job[1]
        location = job[2]
        work_mode = job[3]
        skills = job[4]
        details = job[5]

        # transofrm skills array to string
        skills_str = ','.join(skills)

        # Define values to be inserted
        values = (position, company, location, work_mode, skills_str, details)

        # Execute insert statement
        cursor.execute(insert_query, values)

        # Commit changes to the database
        conn.commit()

In [188]:
# Open Connexion
cursor = conn.cursor()

# save jobs
save_jobs(jobs)

### Close Connection

In [ ]:
# Close the cursor and connection
cursor.close()
conn.close()